In [1]:
# installations
%pip install langchain | tail -n 1
%pip  install langchain-ibm | tail -n 1
%pip install langchain-community | tail -n 1
%pip install ibm-watsonx-ai | tail -n 1
%pip install chromadb | tail -n 1
%pip install tiktoken | tail -n 1
%pip install bs4 | tail -n 1



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Standard libraries
import os
import json
import re
import tempfile
import statistics
import getpass
from typing import List, Dict, Any, Optional

# Third-party
from pydantic import BaseModel, field_validator

# LangChain (core-first to avoid version issues)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.output_parsers import BaseOutputParser

from langchain.memory import ConversationBufferMemory
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents import AgentExecutor
from langchain.prompts import PromptTemplate

from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

# IBM COS
import ibm_boto3
from ibm_botocore.client import Config

from bs4 import SoupStrainer

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
#Setting Up API key and Project ID
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your watsonx.ai Runtime API key (hit enter): ")
}

project_id = getpass.getpass("Please enter your project ID (hit enter): ")



Please enter your watsonx.ai Runtime API key (hit enter):  ········
Please enter your project ID (hit enter):  ········


In [3]:
llm = WatsonxLLM(
    model_id= "ibm/granite-3-8b-instruct", 
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.TEMPERATURE: 0,
        GenParams.MIN_NEW_TOKENS: 5,
        GenParams.MAX_NEW_TOKENS: 250,
        GenParams.STOP_SEQUENCES: ["Human:", "Observation"],
    },
)
IBM_COS_APIKEY = "EsDmUH9cHJS9thi88g0uxtSjasCKMbZ8-TmUXM37opyC"
IBM_COS_RESOURCE_INSTANCE_ID = "crn:v1:bluemix:public:cloud-object-storage:global:a/e2a0be93aad9467c9ff699c583ef8342:eac1dd36-503d-4611-aa11-eea87d23b466::"
IBM_COS_ENDPOINT_URL = "https://s3.us-east.cloud-object-storage.appdomain.cloud"  
BUCKET_NAME = "jump-data"
OBJECT_KEY = "jump_data.json"

def _cos_client():
    return ibm_boto3.client(
        "s3",
        ibm_api_key_id=IBM_COS_APIKEY,
        ibm_service_instance_id=IBM_COS_RESOURCE_INSTANCE_ID,
        config=Config(signature_version="oauth"),
        endpoint_url=IBM_COS_ENDPOINT_URL,
    )

In [ ]:
#prompt template in case you want to ask multiple questions.
template = "Answer the {query} accurately. If you do not know the answer, simply say you do not know."
prompt = PromptTemplate.from_template(template)
#set up a chain with our prompt and our LLM. This allows the generative model to produce a response.
agent = prompt | llm

In [ ]:
#Ask the Agent a question
agent.invoke({"query": "What is the optimal High Jump technique"})

In [ ]:
#Asking another set of Questions
agent.invoke({"query": "What was the takeoff angle of my knee?"})

In [ ]:
#Asking another set of Questions
agent.invoke({"query": "What was the takeoff angle of my knee during my jump?"})

In [ ]:
#Asking another set of Questions
agent.invoke({"query": "If you had access to my jump data via MediaPipe, would you be able to analyze the physical movements?"})

In [5]:
#Websites that data is extracted from... Data Source/knowledge base
urls = [
        "https://en.wikipedia.org/wiki/High_jump",
        "https://coachathletics.com.au/coaching-education/how-to-coach-the-fosbury-flop-drill-progression-for-teaching-high-jump"
    ]

In [6]:

#Loading documents using LangChain WebBaseLoader from urls listed
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
docs_list[0]

Document(metadata={'source': 'https://en.wikipedia.org/wiki/High_jump', 'title': 'High jump - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nHigh jump - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\

In [7]:
#Text Splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

In [8]:
#Embedding models
embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
)

In [9]:
#Storage of embedded documents using Chroma DB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="agentic-rag-chroma",
    embedding=embeddings,
)

In [10]:
#Access Information in vector store
retriever = vectorstore.as_retriever()

In [11]:
# Simple module-level cache so the agent doesn't need to resend big JSON blobs
_JUMP_DATA_CACHE: Dict[str, "JumpData"] = {}
_DEFAULT_KEY = "current"

# Schema mirrors jump_data.json (flat lists)
class JumpData(BaseModel):
    predicted_success: bool
    velocity: List[float]
    hip_height: List[float]
    l_hip_angle: List[float]
    l_knee_angle: List[float]
    l_ankle_angle: List[float]
    r_hip_angle: List[float]
    r_knee_angle: List[float]
    r_ankle_angle: List[float]

    @field_validator("predicted_success", mode="before")
    @classmethod
    def coerce_bool(cls, v):
        if isinstance(v, bool):
            return v
        if isinstance(v, (int, float, str)):
            return str(v).lower() not in ("0", "0.0", "", "false")
        return bool(v)

    def length(self) -> int:
        return len(self.velocity)

    def validate_lengths(self) -> None:
        n = self.length()
        for series in [
            self.hip_height, self.l_hip_angle, self.l_knee_angle, self.l_ankle_angle,
            self.r_hip_angle, self.r_knee_angle, self.r_ankle_angle
        ]:
            if len(series) != n:
                raise ValueError("All series must be the same length as velocity.")

def _require_loaded(key: str = _DEFAULT_KEY) -> JumpData:
    if key not in _JUMP_DATA_CACHE:
        raise ValueError("No jump_data loaded. Call load_jump_data_cos first.")
    return _JUMP_DATA_CACHE[key]

def _summarize_jump(j: JumpData) -> Dict[str, Any]:
    n = j.length()
    if n == 0:
        raise ValueError("jump_data.velocity is empty.")

    idx_peak_hip = max(range(n), key=lambda i: j.hip_height[i])
    idx_max_v = max(range(n), key=lambda i: j.velocity[i])

    if idx_peak_hip > 0:
        l_min_pre = min(range(idx_peak_hip), key=lambda i: j.l_ankle_angle[i])
        r_min_pre = min(range(idx_peak_hip), key=lambda i: j.r_ankle_angle[i])
        plant_idx = l_min_pre if j.l_ankle_angle[l_min_pre] < j.r_ankle_angle[r_min_pre] else r_min_pre
    else:
        plant_idx = 0

    takeoff_idx = min(n - 1, plant_idx + 1)
    approach_window_end = max(1, n // 3)
    approach_max_v = max(j.velocity[:approach_window_end]) if approach_window_end > 0 else j.velocity[0]

    return {
        "length": n,
        "predicted_success": j.predicted_success,
        "idx": {
            "peak_hip": idx_peak_hip,
            "max_velocity": idx_max_v,
            "plant": plant_idx,
            "takeoff": takeoff_idx,
        },
        "metrics": {
            "peak_hip_height": j.hip_height[idx_peak_hip],
            "max_velocity": j.velocity[idx_max_v],
            "approach_max_velocity": approach_max_v,
            "mean_velocity": statistics.fmean(j.velocity),
        },
    }

def _event_slice(j: JumpData, center: int, radius: int = 5) -> Dict[str, Any]:
    n = j.length()
    a = max(0, center - radius)
    b = min(n, center + radius + 1)
    sl = lambda arr: arr[a:b]
    return {
        "frame_start": a,
        "frame_end": b - 1,
        "velocity": sl(j.velocity),
        "hip_height": sl(j.hip_height),
        "l_hip_angle": sl(j.l_hip_angle),
        "l_knee_angle": sl(j.l_knee_angle),
        "l_ankle_angle": sl(j.l_ankle_angle),
        "r_hip_angle": sl(j.r_hip_angle),
        "r_knee_angle": sl(j.r_knee_angle),
        "r_ankle_angle": sl(j.r_ankle_angle),
    }

def _get_key_event_windows(j: JumpData, radius: int = 5) -> Dict[str, Any]:
    s = _summarize_jump(j)
    idx = s["idx"]
    return {
        "indices": idx,
        "windows": {
            "plant": _event_slice(j, idx["plant"], radius),
            "takeoff": _event_slice(j, idx["takeoff"], radius),
            "peak_hip": _event_slice(j, idx["peak_hip"], radius),
        }
    }

def _coaching_feedback(j: JumpData) -> Dict[str, Any]:
    s = _summarize_jump(j)
    idx = s["idx"]
    notes: List[str] = []

    if s["metrics"]["approach_max_velocity"] < 3.5:
        notes.append("Approach speed looks conservative—build rhythm/speed a touch earlier.")
    else:
        notes.append("Approach speed trend is solid—prioritize control into the last two steps.")

    l_ank = j.l_ankle_angle[idx["plant"]]
    r_ank = j.r_ankle_angle[idx["plant"]]
    if min(l_ank, r_ank) > 120:
        notes.append("Limited dorsiflexion at plant—allow more ankle/knee load before takeoff.")
    else:
        notes.append("Good ankle load at plant—keep posture tall as you rise.")

    if idx["peak_hip"] - idx["takeoff"] < 3:
        notes.append("Peak comes very quickly—ensure you’re not cutting the takeoff short.")
    else:
        notes.append("Time-to-peak is reasonable—maintain flight posture and ‘show the hips’.")

    l_knee = j.l_knee_angle[idx["takeoff"]]
    r_knee = j.r_knee_angle[idx["takeoff"]]
    if abs(l_knee - r_knee) > 20:
        notes.append("Knee angles at takeoff look asymmetrical—clean up last step posture and line.")
    else:
        notes.append("Knee symmetry looks okay at takeoff.")

    if not j.predicted_success:
        notes.append("Model flags this as a likely miss—tighten curve and posture into plant.")

    return {
        "summary": s,
        "key_event_windows": _get_key_event_windows(j),
        "notes": notes
    }

@tool
def load_jump_data_cos(bucket_name: Optional[str] = None, key: Optional[str] = None, cache_key: str = _DEFAULT_KEY) -> Dict[str, Any]:
    """Load jump_data.json from IBM COS (bucket/key), validate, cache, and return a light summary."""
    bn = bucket_name or BUCKET_NAME
    kk = key or OBJECT_KEY
    cos = _cos_client()
    obj = cos.get_object(Bucket=bn, Key=kk)
    raw = json.loads(obj["Body"].read().decode("utf-8"))
    data = JumpData(**raw)
    data.validate_lengths()
    _JUMP_DATA_CACHE[cache_key] = data
    s = _summarize_jump(data)
    return {
        "status": "ok",
        "cache_key": cache_key,
        "length": s["length"],
        "predicted_success": s["predicted_success"],
        "indices": s["idx"],
        "metrics": s["metrics"],
    }

@tool
def summarize_jump(jump_data: Optional[Dict[str, Any]] = None, cache_key: str = _DEFAULT_KEY) -> Dict[str, Any]:
    """Return high-level metrics and indices. If jump_data omitted, uses cached."""
    j = JumpData(**jump_data) if jump_data else _require_loaded(cache_key)
    return _summarize_jump(j)

@tool
def get_key_event_windows(jump_data: Optional[Dict[str, Any]] = None, radius: int = 5, cache_key: str = _DEFAULT_KEY) -> Dict[str, Any]:
    """Return slices (±radius) for plant, takeoff, and peak hip height. If jump_data omitted, uses cached."""
    j = JumpData(**jump_data) if jump_data else _require_loaded(cache_key)
    return _get_key_event_windows(j, radius)

@tool
def coaching_feedback(jump_data: Optional[Dict[str, Any]] = None, cache_key: str = _DEFAULT_KEY) -> Dict[str, Any]:
    """Generate heuristic coaching notes. If jump_data omitted, uses cached."""
    j = JumpData(**jump_data) if jump_data else _require_loaded(cache_key)
    return _coaching_feedback(j)

@tool
def kb_search(query: str, k: int = 4) -> Dict[str, Any]:
    """Search the KB retriever and return up to k snippets with sources."""
    try:
        _ = retriever
    except NameError:
        return {"status": "error", "message": "Retriever not initialized."}
    docs = retriever.get_relevant_documents(query)[:k]
    return {
        "results": [
            {"snippet": d.page_content[:600], "source": d.metadata.get("source") or d.metadata.get("url")}
            for d in docs
        ]
    }

tools = [load_jump_data_cos, summarize_jump, get_key_event_windows, coaching_feedback, kb_search]

In [12]:
system_prompt = (
    "You are a Virtual High Jump Coach. Use ONLY the provided tools for numeric metrics "
    "(indices, angles, heights, velocities) from jump_data. Combine tool-derived metrics with "
    "knowledge-base context via kb_search. Do not invent fields in jump_data; treat it as ground truth.\n\n"
    "TOOLS AVAILABLE (names): {tool_names}\n"
    "TOOLS (descriptions & args):\n{tools}\n\n"
    "OUTPUT CONTRACT (STRICT):\n"
    "- On each turn, output exactly ONE JSON object ONLY (no extra text, no code fences).\n"
    "- Allowed keys per turn:\n"
    "  • If taking an action: \"Question\", \"Thought\", \"Action\"\n"
    "  • If done: \"Question\", \"Thought\", \"Final Answer\"\n"
    "- Each tool call MUST be exactly: {\"action\": \"<tool_name>\", \"action_input\": { ... }}\n"
    "- NEVER include an \"Observation\" key; the runtime will supply Observations separately.\n\n"
    "MANDATORY SEQUENCE (exact order across turns):\n"
    "Turn 1 → Action: load_jump_data_cos  with action_input = "
    "{{\"bucket_name\": \"jump-data\", \"key\": \"jump_data.json\"}}\n"
    "Turn 2 → Action: get_key_event_windows with action_input = {{\"radius\": 5}}\n"
    "Turn 3 → Action: summarize_jump       with action_input = {{}}\n"
    "Turn 4 → Action: kb_search            with action_input = "
    "{{\"query\": \"high jump approach velocity and takeoff angles, bar clearance cues, Fosbury flop technique\", \"k\": 4}}\n"
    "Turn 5 → Action: coaching_feedback    with action_input = {{}}\n"
    "Turn 6 → Final Answer: Write clear, actionable coaching guidance. Reference concrete indices "
    "(plant/takeoff/peak) and values (velocity/angles/hip height) from tools, and weave in technique "
    "context from kb_search.\n\n"
    "GENERAL RULES:\n"
    "- After you output your single JSON object, STOP. Wait for the Observation next turn.\n"
    "- Use concise \"Thought\"—just enough to justify the next action.\n"
    "- Do not skip steps or reorder them.\n"
)


In [13]:
human_prompt = """{input}

{agent_scratchpad}

(reminder to always respond in a JSON blob)
"""

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("human", human_prompt),
    ]
).partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

In [16]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

def format_log_to_str(intermediate_steps):
    lines = []
    for action, observation in intermediate_steps:
        lines.append(f'Action: {action.tool} | Input: {action.tool_input}\nObservation: {observation}')
    return "\n".join(lines)

/tmp/wsuser/ipykernel_637/725993922.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [17]:
# --- CoachJSONOutputParser (extended, recommended) ---------------------------
import json, re
from typing import Optional
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.agents import AgentAction, AgentFinish

class CoachJSONOutputParser(BaseOutputParser):
    """Robust parser for your JSON loop:
       - Strips code fences & 'Assistant:' prefix
       - Takes the first balanced {...} block
       - If that JSON lacks Action/Final Answer, tries to SALVAGE an Action object later in the text
       - Returns AgentAction or AgentFinish; raises only for truly malformed Action
    """
    def _clean(self, text: str) -> str:
        t = text.strip()
        # strip code fences
        if t.startswith("```"):
            t = re.sub(r"^```(?:json)?\s*", "", t, flags=re.IGNORECASE)
            t = re.sub(r"\s*```$", "", t)
        # strip leading Assistant: label if present
        t = re.sub(r'^\s*Assistant:\s*', '', t)
        # normalize quotes
        return t.replace("“", '"').replace("”", '"').replace("’", "'").replace("‘", "'")

    def _extract_balanced_json(self, text: str) -> Optional[str]:
        start = text.find("{")
        if start == -1:
            return None
        i, depth, in_str, esc = start, 0, False, False
        while i < len(text):
            ch = text[i]
            if in_str:
                if esc:
                    esc = False
                elif ch == "\\":
                    esc = True
                elif ch == '"':
                    in_str = False
            else:
                if ch == '"':
                    in_str = True
                elif ch == "{":
                    depth += 1
                elif ch == "}":
                    depth -= 1
                    if depth == 0:
                        return text[start:i+1]
            i += 1
        return None

    def _extract_balanced_from(self, s: str, start_idx: int) -> Optional[str]:
        """Find next balanced {...} starting at/after start_idx."""
        brace = s.find("{", start_idx)
        if brace == -1:
            return None
        i, depth, in_str, esc = brace, 0, False, False
        while i < len(s):
            ch = s[i]
            if in_str:
                if esc:
                    esc = False
                elif ch == "\\":
                    esc = True
                elif ch == '"':
                    in_str = False
            else:
                if ch == '"':
                    in_str = True
                elif ch == "{":
                    depth += 1
                elif ch == "}":
                    depth -= 1
                    if depth == 0:
                        return s[brace:i+1]
            i += 1
        return None

    def parse(self, text: str):
        t = self._clean(text)

        # 1) Try first balanced JSON block
        block = self._extract_balanced_json(t) or t
        try:
            data = json.loads(block)
        except Exception:
            # Can't parse: return the raw as final answer to avoid hard crash
            return AgentFinish(return_values={"output": text}, log=text)

        action_field = data.get("Action")

        # 2) If provided Final Answer (no/empty Action), finish
        if "Final Answer" in data and (not action_field or action_field == {} or (isinstance(action_field, str) and not action_field.strip())):
            return AgentFinish(return_values={"output": data["Final Answer"]}, log=text)

        # 3) Proper Action present
        if isinstance(action_field, dict):
            tool = action_field.get("action")
            tool_input = action_field.get("action_input", {})
            if not tool:
                raise ValueError("Missing 'action' in Action object.")
            if not isinstance(tool_input, dict):
                raise ValueError("'action_input' must be a JSON object (dict).")
            return AgentAction(tool=tool, tool_input=tool_input, log=text)

        # 4) Salvage: search for an "Action": { ... } later in the text
        m = re.search(r'"Action"\s*:\s*', t)
        if m:
            sub_json = self._extract_balanced_from(t, m.end())
            if sub_json:
                try:
                    af = json.loads(sub_json)
                    tool = af.get("action")
                    tool_input = af.get("action_input", {})
                    if tool and isinstance(tool_input, dict):
                        return AgentAction(tool=tool, tool_input=tool_input, log=text)
                except Exception:
                    pass

        # 5) If still provided a Final Answer somewhere in data
        if "Final Answer" in data:
            return AgentFinish(return_values={"output": data["Final Answer"]}, log=text)

        # 6) Nothing usable
        raise ValueError("Expected an Action object or a Final Answer.")

In [18]:
chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x.get("intermediate_steps", [])),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | llm
    | CoachJSONOutputParser()
)

agent_executor = AgentExecutor(
    agent=chain,
    tools=tools,
    handle_parsing_errors=True,
    verbose=True,
    memory=memory,
)


In [26]:
#We are now able to ask the agent questions.
agent_executor.invoke({"input":"How can I improve my jump?"})



> Entering new AgentExecutor chain...


KeyError: 'Input to ChatPromptTemplate is missing variables {\'"action"\'}.  Expected: [\'"action"\', \'agent_scratchpad\', \'input\'] Received: [\'input\', \'intermediate_steps\', \'agent_scratchpad\', \'chat_history\']\nNote: if you intended {"action"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{"action"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [20]:
# 1) Load once from COS
print(load_jump_data_cos.invoke({"bucket_name": BUCKET_NAME, "key": OBJECT_KEY}))

# 2) Confirm it’s cached
print("Cache keys:", list(_JUMP_DATA_CACHE.keys()))

# 3) Use cached data with other tools (no jump_data arg needed)
print(summarize_jump.invoke({}))
print(get_key_event_windows.invoke({"radius": 5}))
print(coaching_feedback.invoke({}))


{'status': 'ok', 'cache_key': 'current', 'length': 109, 'predicted_success': False, 'indices': {'peak_hip': 84, 'max_velocity': 96, 'plant': 79, 'takeoff': 80}, 'metrics': {'peak_hip_height': 88.4783150271207, 'max_velocity': 13.931530982124197, 'approach_max_velocity': 3.1044334443569817, 'mean_velocity': 3.959227001901344}}
Cache keys: ['current']
{'length': 109, 'predicted_success': False, 'idx': {'peak_hip': 84, 'max_velocity': 96, 'plant': 79, 'takeoff': 80}, 'metrics': {'peak_hip_height': 88.4783150271207, 'max_velocity': 13.931530982124197, 'approach_max_velocity': 3.1044334443569817, 'mean_velocity': 3.959227001901344}}
{'indices': {'peak_hip': 84, 'max_velocity': 96, 'plant': 79, 'takeoff': 80}, 'windows': {'plant': {'frame_start': 74, 'frame_end': 84, 'velocity': [2.4709725961171003, 2.614543627170265, 2.7991455306106836, 3.054949157345313, 3.45068451766055, 4.0434263208928165, 4.8211224552692755, 5.7176534602599975, 6.642699231692288, 7.481545710129273, 8.128434764061153], '

In [21]:
import json, re
from typing import Any, Dict, List, Tuple, Optional

from langchain_core.agents import AgentAction, AgentFinish
class CoachJSONOutputParser(BaseOutputParser):
    """Robust parser for your JSON loop."""
    def _strip_fences_and_normalize(self, text: str) -> str:
        t = text.strip()
        if t.startswith("```"):
            t = re.sub(r"^```(?:json)?\s*", "", t, flags=re.IGNORECASE)
            t = re.sub(r"\s*```$", "", t)
        return t.replace("“", '"').replace("”", '"').replace("’", "'").replace("‘", "'")

    def _extract_balanced_json(self, text: str) -> Optional[str]:
        start = text.find("{")
        if start == -1:
            return None
        i, depth, in_str, esc = start, 0, False, False
        while i < len(text):
            ch = text[i]
            if in_str:
                if esc: esc = False
                elif ch == "\\": esc = True
                elif ch == '"': in_str = False
            else:
                if ch == '"': in_str = True
                elif ch == "{": depth += 1
                elif ch == "}":
                    depth -= 1
                    if depth == 0:
                        return text[start:i+1]
            i += 1
        return None

    def parse(self, text: str):
        cleaned = self._strip_fences_and_normalize(text)
        candidate = self._extract_balanced_json(cleaned) or cleaned
        try:
            data = json.loads(candidate)
        except Exception:
            return AgentFinish(return_values={"output": text}, log=text)

        action_field = data.get("Action")
        if "Final Answer" in data and (action_field in (None, "", {}) or (isinstance(action_field, str) and not action_field.strip())):
            return AgentFinish(return_values={"output": data["Final Answer"]}, log=text)
        if isinstance(action_field, dict):
            tool = action_field.get("action")
            tool_input = action_field.get("action_input", {})
            if not tool:
                raise ValueError("Missing 'action' in Action object.")
            if not isinstance(tool_input, dict):
                raise ValueError("'action_input' must be a JSON object (dict).")
            return AgentAction(tool=tool, tool_input=tool_input, log=text)
        if "Final Answer" in data:
            return AgentFinish(return_values={"output": data["Final Answer"]}, log=text)
        raise ValueError("Expected an Action object or a Final Answer.")

# assumes your CoachJSONOutputParser class is already defined in the notebook

def run_parser_self_tests_for_current_parser(parser) -> List[Tuple[str, bool, str]]:
    """
    Runs a suite of cases against your CURRENT CoachJSONOutputParser (the minimal version).
    Returns a list of (case_name, passed_bool, details_str) and prints a summary.
    NOTE: This parser does NOT salvage actions later in the text, so that test expects an ERROR.
    """
    # Helper expectations
    def exp_action(tool: str, contains: Dict[str, Any] | None = None):
        return {"type": "action", "tool": tool, "contains": contains or {}}
    def exp_finish():
        return {"type": "finish"}
    def exp_error():
        return {"type": "error"}

    tests: List[Tuple[str, str, Dict[str, Any]]] = [
        (
            "valid_action_minimal",
            '{ "Question":"Q", "Thought":"T", "Action": {"action":"load_jump_data_cos","action_input":{"bucket_name":"jump-data","key":"jump_data.json"}} }',
            exp_action("load_jump_data_cos", {"bucket_name": "jump-data"})
        ),
        (
            "valid_final_answer",
            '{ "Question":"Q", "Thought":"T", "Final Answer":"All good." }',
            exp_finish()
        ),
        (
            "codefenced_json",
            "```json\n{ \"Question\":\"Q\", \"Thought\":\"T\", \"Action\": {\"action\":\"summarize_jump\",\"action_input\":{}} }\n```",
            exp_action("summarize_jump")
        ),
        (
            "assistant_prefixed",
            'Assistant:\n{ "Question":"Q", "Thought":"T", "Action": {"action":"get_key_event_windows","action_input":{"radius":5}} }',
            # This parser finds the first '{' anywhere, so it should still PASS
            exp_action("get_key_event_windows", {"radius": 5})
        ),
        (
            "observation_leak_after_json",
            '{ "Question":"Q", "Thought":"T", "Action": {"action":"coaching_feedback","action_input":{}} }\nObservation: {"status":"ok"}',
            exp_action("coaching_feedback")
        ),
        (
            "multi_json_take_first",
            '{ "Question":"Q1", "Thought":"T1", "Action": {"action":"kb_search","action_input":{"query":"fosbury flop","k":2}} }\n\n{ "Question":"Q2","Thought":"T2","Final Answer":"done"}',
            exp_action("kb_search", {"query": "fosbury flop"})
        ),
        (
            "salvage_action_later_in_text",
            '{ "Question":"Q", "Thought":"T", "Note": "no action here" }\nSome text...\n"Action": {"action":"summarize_jump","action_input":{}}',
            # This parser does NOT salvage a later Action, so we expect an ERROR
            exp_error()
        ),
        (
            "malformed_trailing_dict",
            '{ "Question":"Q", "Thought":"T", "Action": {"action":"summarize_jump","action_input":{}} }  {"status":"ok"}',
            exp_action("summarize_jump")
        ),
        (
            "bad_action_object",
            '{ "Question":"Q", "Thought":"T", "Action": {"action_input": {}} }',  # missing "action"
            exp_error()
        ),
        (
            "finish_without_action_object",
            '{ "Question":"Q", "Thought":"T", "Final Answer": "Done." }',
            exp_finish()
        ),
    ]

    results: List[Tuple[str, bool, str]] = []
    width = max(len(n) for n,_,_ in tests) + 2
    print("Parser self-tests (current minimal CoachJSONOutputParser):")
    for name, text, expect in tests:
        try:
            out = parser.parse(text)
            if isinstance(out, AgentAction):
                want = expect.get("type") == "action"
                tool_ok = out.tool == expect.get("tool")
                contains_ok = all(
                    (k in out.tool_input and out.tool_input[k] == v)
                    for k, v in expect.get("contains", {}).items()
                )
                passed = want and tool_ok and contains_ok
                details = f"Got AgentAction(tool={out.tool}, tool_input={out.tool_input})"
            elif isinstance(out, AgentFinish):
                passed = expect.get("type") == "finish"
                details = f"Got AgentFinish(output={out.return_values.get('output')!r})"
            else:
                passed = False
                details = f"Unexpected return type: {type(out)}"
        except Exception as e:
            passed = (expect.get("type") == "error")
            details = f"Raised: {type(e).__name__}: {e}"

        results.append((name, passed, details))
        print(f"{name.ljust(width)} {'PASS' if passed else 'FAIL'} - {details}")

    total = len(results)
    passed = sum(1 for _, ok, _ in results if ok)
    print(f"\nSummary: {passed}/{total} tests passed.\n")
    return results
parser = CoachJSONOutputParser()
_ = run_parser_self_tests_for_current_parser(parser)


Parser self-tests (current minimal CoachJSONOutputParser):
valid_action_minimal           PASS - Got AgentAction(tool=load_jump_data_cos, tool_input={'bucket_name': 'jump-data', 'key': 'jump_data.json'})
valid_final_answer             PASS - Got AgentFinish(output='All good.')
codefenced_json                PASS - Got AgentAction(tool=summarize_jump, tool_input={})
assistant_prefixed             PASS - Got AgentAction(tool=get_key_event_windows, tool_input={'radius': 5})
observation_leak_after_json    PASS - Got AgentAction(tool=coaching_feedback, tool_input={})
multi_json_take_first          PASS - Got AgentAction(tool=kb_search, tool_input={'query': 'fosbury flop', 'k': 2})
salvage_action_later_in_text   PASS - Raised: ValueError: Expected an Action object or a Final Answer.
malformed_trailing_dict        PASS - Got AgentAction(tool=summarize_jump, tool_input={})
bad_action_object              PASS - Raised: ValueError: Missing 'action' in Action object.
finish_without_action_object 

In [22]:
print(load_jump_data_cos.invoke({"bucket_name": BUCKET_NAME, "key": OBJECT_KEY}))
print("Cache keys:", list(_JUMP_DATA_CACHE.keys()))


{'status': 'ok', 'cache_key': 'current', 'length': 109, 'predicted_success': False, 'indices': {'peak_hip': 84, 'max_velocity': 96, 'plant': 79, 'takeoff': 80}, 'metrics': {'peak_hip_height': 88.4783150271207, 'max_velocity': 13.931530982124197, 'approach_max_velocity': 3.1044334443569817, 'mean_velocity': 3.959227001901344}}
Cache keys: ['current']


In [23]:
print(summarize_jump.invoke({}))
print(get_key_event_windows.invoke({"radius": 5}))
print(coaching_feedback.invoke({}))


{'length': 109, 'predicted_success': False, 'idx': {'peak_hip': 84, 'max_velocity': 96, 'plant': 79, 'takeoff': 80}, 'metrics': {'peak_hip_height': 88.4783150271207, 'max_velocity': 13.931530982124197, 'approach_max_velocity': 3.1044334443569817, 'mean_velocity': 3.959227001901344}}
{'indices': {'peak_hip': 84, 'max_velocity': 96, 'plant': 79, 'takeoff': 80}, 'windows': {'plant': {'frame_start': 74, 'frame_end': 84, 'velocity': [2.4709725961171003, 2.614543627170265, 2.7991455306106836, 3.054949157345313, 3.45068451766055, 4.0434263208928165, 4.8211224552692755, 5.7176534602599975, 6.642699231692288, 7.481545710129273, 8.128434764061153], 'hip_height': [72.33239824761594, 75.79461329951657, 78.5658762432197, 80.45403044940595, 81.56321967662161, 82.4415282666617, 83.6690718435786, 85.34425944491578, 87.053573348797, 88.2457685952309, 88.4783150271207], 'l_hip_angle': [153.84738744289183, 36.17010906820837, 161.71309133980193, 52.88324323155756, 148.87055876706754, 65.25879632592374, 15

In [24]:
print(kb_search.invoke({"query": "Fosbury flop takeoff angle cues", "k": 3}))


/tmp/wsuser/ipykernel_637/2221365152.py:193: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)[:k]


{'results': [{'snippet': "Approach[edit]\nSpanish jumper Ruth Beitia approaching the bar from an angle\nFor a Fosbury Flop, depending on the athlete's jump foot, they start on the right or left of the high jump mat, placing their jump foot farthest away from the mat. They take an eight- to ten-step approach, with the first three to five steps being in a straight line and the last five being on a curve. Athletes generally mark their approach in order to find as much consistency as possible.\nThe approach run can be more important than the takeoff. If a high jumper runs with bad timing or without enough aggression, clearing", 'source': 'https://en.wikipedia.org/wiki/High_jump'}, {'snippet': 'Takeoff[edit]\nThe takeoff can be double-arm or single-arm. In both cases, the plant foot should be the foot farthest from the bar, angled towards the opposite back corner of the mat, as they drive up the knee on their non-takeoff leg. This is accompanied by a one- or two-arm swing while driving the 